In [ ]:
from keras.callbacks import ModelCheckpoint, Callback

In [ ]:
class LossValidateCallback(Callback):

    def __init__(self, batch_generator, results_file):
        self.batch_generator = batch_generator

        basedir = os.path.dirname(results_file)
        if not os.path.exists(basedir):
            os.makedirs(basedir)
        self.results_file = results_file

    @staticmethod
    def IOU_loss(y_true, y_false):
        def IOU_calc(y_true, y_false, smooth=1.):
            y_true_f = y_true.flatten()
            y_false_f = y_false.flatten()
            intersection = np.sum(y_true_f * y_false_f)
            union = np.sum(y_true_f) + np.sum(y_false_f) - intersection
            return (intersection + smooth) / (union + smooth)

        return 1 - IOU_calc(y_true, y_false)

    def on_epoch_end(self, epoch, logs=None):
        train_batch, val_batch = self.batch_generator(VAL_BATCH)
        train_imgs, train_masks = train_batch
        val_imgs, val_masks = val_batch
        train_results = self.model.predict(train_imgs)
        val_results = self.model.predict(val_imgs)

        # change categorical
        train_losses = [
            LossValidateCallback.IOU_loss(
                mask_to_categorical(pair[0]), pair[1]
            )
            for pair in zip(train_masks, train_results)
        ]
        average_train_loss = np.average(train_losses)
        std_train_loss = np.std(train_losses)

        # change categorical
        val_losses = [
            LossValidateCallback.IOU_loss(
                mask_to_categorical(pair[0]), pair[1]
            )
            for pair in zip(val_masks, val_results)
        ]
        average_val_loss = np.average(val_losses)
        std_val_loss = np.std(val_losses)

        # change categorical
        train_size = len(train_masks)
        batch_train_loss = LossValidateCallback.IOU_loss(
            mask_to_categorical(train_masks, train_size), train_results
        )

        # change categorical
        val_size = len(val_masks)
        batch_val_loss = LossValidateCallback.IOU_loss(
            mask_to_categorical(val_masks, val_size), val_results
        )

        eval_train_loss, _ = self.model.evaluate(
            train_imgs, mask_to_categorical(train_masks, train_size)
        )
        eval_val_loss, _ = self.model.evaluate(
            val_imgs, mask_to_categorical(val_masks, val_size)
        )

        text = '{0}, {1}, {2}, '.format(epoch, eval_train_loss, eval_val_loss)
        text += '{0}, {1}, '.format(batch_train_loss, batch_val_loss)
        text += '{0}, {1}, {2}, {3}\n'.format(
            average_train_loss, std_train_loss,
            average_val_loss, std_val_loss
        )
        if not os.path.exists(self.results_file):
            with open(self.results_file, 'w') as file:
                columns = 'epoch, eval_train_loss, eval_validation_loss, '
                columns += 'batch_train_loss, batch_val_loss, '
                columns += 'batch_stats_train_avg_loss, batch_stats_train_std_loss, '
                columns += 'batch_stats_val_avg_loss, batch_stats_val_std_loss\n'
                file.writelines(columns)
            
        with open(self.results_file, 'a') as file:
            file.writelines(text)